# UK - Anbindung Paid Themen

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):
- Die Paid Data Transformatation von Meta


---
* QUELLEN:  
- datif_dz_{}.`02_cleaned_uk_facebook`.`018_meta_ads_c_kkc_some_current_view`

* ZIEL:  
- datif_pz_uk_{}.03_transformed.meta_paid_daily
- datif_pz_uk_{}.03_transformed.meta_paid_total

---
* Versionen (aktuelle immer oben):
- 18.09.2025 Justin Stange-Heiduk: Add OrganicID
- 20.08.2025 Justin Stange-Heiduk: init


# 1. Initialnotebooks & Libraries

In [0]:
%run ../../common/nb_init

## 03-transformed

In [0]:
target_schema_name = "03_transformed"
target_path = "funnel"

### Meta daily

In [0]:
spark.sql(
    f"""
    CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.meta_paid_daily AS
    SELECT
      First(CAST(DATE_FORMAT(Date, 'yyyy-MM-dd') as DATE)) as Date,
      First(CAST(Ad_ID__Facebook_Ads AS STRING)) as AdID,
      First(CAST(Campaign_Name__Facebook_Ads AS STRING)) as CampaignName,
      First(CAST(Ad_Set_Name__Facebook_Ads AS STRING)) as AdSetName,
      First(CAST(Ad_Name__Facebook_Ads AS STRING)) as AdName,
      case
             when AdName like "%!%"
             THEN REGEXP_EXTRACT(AdName, '!([^!]+)$', 1)
             ELSE 'Dark Post'
            END AS OrganicID,
      Sum(COALESCE(round(CAST(Amount_Spent__Facebook_Ads AS Double),2),0)) as Amount_Spend,
      Sum(COALESCE(CAST(Impressions__Facebook_Ads AS INTEGER),0)) as Impressions,
      Sum(COALESCE(CAST(Clicks_all__Facebook_Ads AS INTEGER),0)) as Clicks,
      Sum(COALESCE(CAST(Link_Clicks__Facebook_Ads AS INTEGER),0)) as Link_Clicks,
      Sum(COALESCE(CAST(Post_Saves__Facebook_Ads AS INTEGER),0)) as Post_Saves,
      Sum(COALESCE(CAST(Post_Shares__Facebook_Ads AS INTEGER),0)) as Post_Shares,
      Sum(COALESCE(CAST(Post_Comments__Facebook_Ads AS INTEGER),0)) as Comments,
      Sum(COALESCE(CAST(Post_Reactions__Facebook_Ads AS INTEGER),0)) as Reactions,
      Sum(COALESCE(CAST(n_3_Second_Video_Views__Facebook_Ads AS INTEGER),0)) as Video_View,
      COALESCE(
        (0.4 * Comments + 
         0.1 * Post_Shares +
         0.3 * Link_Clicks + 
         0.2 * Reactions
        ),0) As WeightedEngagements,
      COALESCE(
        (Comments + 
         Post_Shares +
         Link_Clicks + 
         Reactions
        ),0) As Engagements
    FROM datif_dz_{env}.`02_cleaned_uk_facebook`.`018_meta_ads_c_kkc_some_current_view`
    GROUP BY Ad_ID__Facebook_Ads, Date
    """
)

### Meta Total

In [0]:
%python
df_daily = spark.read.table(f"datif_pz_uk_{env}.03_transformed.meta_paid_daily")

# Es muss hier nach ADName aggregiert werden weil es Paid Posts gibt die zu einen Organic Post gehören, diese Paid Posts haben unterschiedliche ADIDs aber die gleiche ADName
df_total = (
    df_daily.groupBy("ADName").agg(
        F.first("AdID").alias("ID"),
        F.first("OrganicID").alias("OrganicID"),
        F.min("Date").alias("CreatedDate").cast('date'),
        F.first("CampaignName").alias("CampaignName"),
        F.first("AdSetName").alias("AdSetName"),
        # F.first("AdName").alias("AdName"),
        F.round(F.sum("Amount_Spend"),2).alias("TotalSpend").cast('Double'),
        F.sum("Impressions").alias("TotalImpressions").cast('integer'),
        F.sum("Clicks").alias("TotalClicks").cast('integer'),
        F.sum("Link_Clicks").alias("TotalLinkClicks").cast('integer'),
        F.sum("Post_Saves").alias("TotalPostSaves").cast('integer'),
        F.sum("Post_Shares").alias("TotalPostShares").cast('integer'),
        F.sum("Comments").alias("TotalPostComments").cast('integer'),
        F.sum("Reactions").alias("TotalReactions").cast('integer'),
        F.sum("Video_View").alias("TotalVideoView").cast('integer'),
        F.sum("WeightedEngagements").alias("TotalWeightedEngagements").cast('integer'),
        F.sum("Engagements").alias("TotalEngagements").cast('integer')
    ) # .drop("ADName")
    .withColumn(
        "EngagementRatings",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    F.col("TotalWeightedEngagements") / F.col("TotalImpressions")
                ), 3
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CPM",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    (F.col("TotalSpend") / F.col("TotalImpressions")) * 1000
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CPC",
        F.coalesce(
            F.round(
                F.when(F.col("TotalClicks") == 0, 0).otherwise(
                    F.col("TotalSpend") / F.col("TotalClicks")
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CTR",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    F.col("TotalClicks") / F.col("TotalImpressions")
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CPV",
        F.coalesce(
            F.round(
                F.when(F.col("TotalVideoView") == 0, 0).otherwise(
                    F.col("TotalSpend") / F.col("TotalVideoView")
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "VideoViewRate",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    F.col("TotalVideoView") / F.col("TotalImpressions")
                ), 4
            ),
            F.lit(0)
        )
    )
    .drop("AdSetName", "Clicks", "Link_Clicks", "Post_Saves", "Post_Shares", "Comments", "Reactions", "Video_View")
)

fn_overwrite_table(df_source=df_total, target_schema_name=target_schema_name, target_table_name="meta_paid_total", target_path=target_path)